<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_whisper_medium_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-etxe3a0y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-etxe3a0y
  Resolved https://github.com/huggingface/transformers to commit cf62539a294ebafdd94f16d443e84fd9a6f7fea0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8118060 sha256=744e016df7d7732b556ad57d9095c3e0ee6c761594e587f05605f4ac32e2ef3d
  Stored in directory: /tmp/pip-ephem-wheel-cache-97k3f08o/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [5]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [6]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 666
})

In [9]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [10]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [01:09<00:00, 21.9MiB/s]


In [11]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [12]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [13]:
text

,0
0,na primjer omogućila je mnogim talentiranim lj...
1,potovani predsjedavajui ribarstvo je vana gosp...
2,nastavlja na temelju ovog europskog projekta k...
3,gospoo predsjedavajua dananje izvjee o europsk...
4,bitno je doprinijeti većem povjerenju potrošač...
...,...
661,radi se o čestim blokadama vlasti napuštanju s...
662,napad islamistikih militanata u sjevernoj nige...
663,ne smijemo ostati slijepi na činjenicu da i eu...
664,u mnogim zemljama ako niste uhvaćeni u počinje...


In [14]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [15]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
...,...,...,...,...,...
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000
664,male,u mnogim zemljama ako niste uhvaćeni u počinje...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.000274658203125, 0.00036...",16000


In [16]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
5,male,takva je pozicija neizbjena ne samo zbog injen...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.007781982421875, -0.009033203125, -0.00741...",16000
6,male,uloga europe mora i dalje biti promicanje ljud...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004669189453125, -0.00543212890625, -0.003...",16000
...,...,...,...,...,...
658,male,stoga sam uvjeren da upravo visoki standardi z...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009552001953125, -0.018829345703125, -0.01...",16000
659,male,od trenutka pristupanja hrvatske u punopravno ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00042724609375, 0.005462646484375, 0.00509...",16000
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000


In [17]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
8,female,dakle sankcije treba uzeti kao mjere koje nisu...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000762939453125, -0.0001...",16000
9,female,ako se ovako nastavi stradat će proizvodnja hr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, 0.000152587890625, 0.0006...",16000
11,female,za ostvarivanje ambicioznih ciljeva potrebno j...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001434326171875, 0.0003662109375, -0.000732...",16000
...,...,...,...,...,...
653,female,kao zastupnica iz hrvatske a svjesna alarmantn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000640869140625, 0.00030...",16000
657,female,niti jedna drava lanica europske unije nije do...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002655029296875, -0.0084228515625, -0.0046...",16000
660,female,kolegice vi jako dobro znate da neke države čl...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00048828125, -0.0006103515625, -0.00018310...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000


In [18]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr")
    return result["text"]

In [19]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [20]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 1.0
Processed 2 files, WER: 0.3793103448275862
Processed 3 files, WER: 0.5142857142857142
Processed 4 files, WER: 0.5434782608695652
Processed 5 files, WER: 0.3684210526315789
Processed 6 files, WER: 0.575
Processed 7 files, WER: 0.6
Processed 8 files, WER: 0.20689655172413793
Processed 9 files, WER: 0.23076923076923078
Processed 10 files, WER: 0.25
Processed 11 files, WER: 1.0
Processed 12 files, WER: 0.4375
Processed 13 files, WER: 0.4
Processed 14 files, WER: 0.3
Processed 15 files, WER: 0.3333333333333333
Processed 16 files, WER: 0.43548387096774194
Processed 17 files, WER: 0.48148148148148145
Processed 18 files, WER: 0.2857142857142857
Processed 19 files, WER: 0.14285714285714285
Processed 20 files, WER: 0.23809523809523808
Processed 21 files, WER: 1.0
Processed 22 files, WER: 0.5
Processed 23 files, WER: 0.48484848484848486
Processed 24 files, WER: 0.18181818181818182
Processed 25 files, WER: 0.2222222222222222
Processed 26 files, WER: 0.5555555555555556
P

In [20]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.14285714285714285
Processed 2 files, WER: 0.3333333333333333
Processed 3 files, WER: 0.30303030303030304
Processed 4 files, WER: 0.3333333333333333
Processed 5 files, WER: 0.2
Processed 6 files, WER: 0.16666666666666666
Processed 7 files, WER: 0.35
Processed 8 files, WER: 0.4166666666666667
Processed 9 files, WER: 0.20833333333333334
Processed 10 files, WER: 1.0
Processed 11 files, WER: 0.9743589743589743
Processed 12 files, WER: 0.38235294117647056
Processed 13 files, WER: 0.2777777777777778
Processed 14 files, WER: 0.2962962962962963
Processed 15 files, WER: 0.32
Processed 16 files, WER: 0.23529411764705882
Processed 17 files, WER: 0.3939393939393939
Processed 18 files, WER: 0.17391304347826086
Processed 19 files, WER: 0.5454545454545454
Processed 20 files, WER: 0.2413793103448276
Processed 21 files, WER: 0.19047619047619047
Processed 22 files, WER: 0.17647058823529413
Processed 23 files, WER: 1.0
Processed 24 files, WER: 0.2727272727272727
Processed 25 file